# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [30]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [31]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [32]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    vocab = sorted(set(text))
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [33]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    return { 
        '.' : '<PERIOD>', ',' : '<COMMA>', '"':'<QUOTATION_MARK>', ';' : '<SEMICOLON>',
        '!' : '<EXCLAMATION_MARK>', '?' : '<QUESTION_MARK>', '(' : '<LEFT_PARENTHESES>',
        ')' : '<RIGHT_PARENTHESES>', '--' : '<DASH>', '\n' : '<RETURN>'
    }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [34]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [35]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [57]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [58]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    return inputs, targets, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [59]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    def build_cell(lstm_size):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        # drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return lstm
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size) for _ in range(5)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name='initial_state')
    
    return cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [60]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [61]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name='final_state')
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [62]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    inputs = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, inputs)
    
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn=None)
    
    return logits, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [63]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # Solution from the Slack Team
    # This was really confusing me
    n_batches = int(len(int_text) / (batch_size * seq_length))

    # Drop the last few characters to make only full batches
    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.roll(xdata, -1)
    
    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)

    return np.array(list(zip(x_batches, y_batches)))

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [68]:
# Number of Epochs
num_epochs = 1000
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 256
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 100
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 5


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [70]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/21   train_loss = 8.822
Epoch   0 Batch    5/21   train_loss = 6.413
Epoch   0 Batch   10/21   train_loss = 6.640
Epoch   0 Batch   15/21   train_loss = 6.665
Epoch   0 Batch   20/21   train_loss = 6.799
Epoch   1 Batch    4/21   train_loss = 6.207
Epoch   1 Batch    9/21   train_loss = 6.139
Epoch   1 Batch   14/21   train_loss = 6.177
Epoch   1 Batch   19/21   train_loss = 6.273
Epoch   2 Batch    3/21   train_loss = 6.253
Epoch   2 Batch    8/21   train_loss = 6.220
Epoch   2 Batch   13/21   train_loss = 6.417
Epoch   2 Batch   18/21   train_loss = 6.260
Epoch   3 Batch    2/21   train_loss = 6.183
Epoch   3 Batch    7/21   train_loss = 6.252
Epoch   3 Batch   12/21   train_loss = 6.107
Epoch   3 Batch   17/21   train_loss = 6.153
Epoch   4 Batch    1/21   train_loss = 6.281
Epoch   4 Batch    6/21   train_loss = 6.169
Epoch   4 Batch   11/21   train_loss = 6.128
Epoch   4 Batch   16/21   train_loss = 6.348
Epoch   5 Batch    0/21   train_loss = 6.258
Epoch   5 

Epoch  43 Batch   12/21   train_loss = 5.882
Epoch  43 Batch   17/21   train_loss = 5.936
Epoch  44 Batch    1/21   train_loss = 6.068
Epoch  44 Batch    6/21   train_loss = 5.961
Epoch  44 Batch   11/21   train_loss = 5.940
Epoch  44 Batch   16/21   train_loss = 6.126
Epoch  45 Batch    0/21   train_loss = 6.008
Epoch  45 Batch    5/21   train_loss = 5.979
Epoch  45 Batch   10/21   train_loss = 5.976
Epoch  45 Batch   15/21   train_loss = 5.957
Epoch  45 Batch   20/21   train_loss = 6.010
Epoch  46 Batch    4/21   train_loss = 5.907
Epoch  46 Batch    9/21   train_loss = 5.919
Epoch  46 Batch   14/21   train_loss = 5.947
Epoch  46 Batch   19/21   train_loss = 5.992
Epoch  47 Batch    3/21   train_loss = 5.972
Epoch  47 Batch    8/21   train_loss = 5.957
Epoch  47 Batch   13/21   train_loss = 6.021
Epoch  47 Batch   18/21   train_loss = 5.937
Epoch  48 Batch    2/21   train_loss = 5.944
Epoch  48 Batch    7/21   train_loss = 6.038
Epoch  48 Batch   12/21   train_loss = 5.832
Epoch  48 

Epoch  87 Batch    3/21   train_loss = 5.490
Epoch  87 Batch    8/21   train_loss = 5.443
Epoch  87 Batch   13/21   train_loss = 5.536
Epoch  87 Batch   18/21   train_loss = 5.477
Epoch  88 Batch    2/21   train_loss = 5.519
Epoch  88 Batch    7/21   train_loss = 5.538
Epoch  88 Batch   12/21   train_loss = 5.370
Epoch  88 Batch   17/21   train_loss = 5.443
Epoch  89 Batch    1/21   train_loss = 5.528
Epoch  89 Batch    6/21   train_loss = 5.438
Epoch  89 Batch   11/21   train_loss = 5.461
Epoch  89 Batch   16/21   train_loss = 5.595
Epoch  90 Batch    0/21   train_loss = 5.464
Epoch  90 Batch    5/21   train_loss = 5.477
Epoch  90 Batch   10/21   train_loss = 5.381
Epoch  90 Batch   15/21   train_loss = 5.367
Epoch  90 Batch   20/21   train_loss = 5.417
Epoch  91 Batch    4/21   train_loss = 5.257
Epoch  91 Batch    9/21   train_loss = 5.268
Epoch  91 Batch   14/21   train_loss = 5.176
Epoch  91 Batch   19/21   train_loss = 5.326
Epoch  92 Batch    3/21   train_loss = 5.205
Epoch  92 

Epoch 130 Batch   15/21   train_loss = 4.025
Epoch 130 Batch   20/21   train_loss = 4.081
Epoch 131 Batch    4/21   train_loss = 3.951
Epoch 131 Batch    9/21   train_loss = 4.064
Epoch 131 Batch   14/21   train_loss = 3.785
Epoch 131 Batch   19/21   train_loss = 4.064
Epoch 132 Batch    3/21   train_loss = 3.943
Epoch 132 Batch    8/21   train_loss = 3.895
Epoch 132 Batch   13/21   train_loss = 3.910
Epoch 132 Batch   18/21   train_loss = 3.958
Epoch 133 Batch    2/21   train_loss = 4.039
Epoch 133 Batch    7/21   train_loss = 3.939
Epoch 133 Batch   12/21   train_loss = 3.873
Epoch 133 Batch   17/21   train_loss = 3.983
Epoch 134 Batch    1/21   train_loss = 4.071
Epoch 134 Batch    6/21   train_loss = 3.875
Epoch 134 Batch   11/21   train_loss = 3.965
Epoch 134 Batch   16/21   train_loss = 4.027
Epoch 135 Batch    0/21   train_loss = 4.005
Epoch 135 Batch    5/21   train_loss = 3.988
Epoch 135 Batch   10/21   train_loss = 3.895
Epoch 135 Batch   15/21   train_loss = 3.939
Epoch 135 

Epoch 174 Batch    6/21   train_loss = 3.465
Epoch 174 Batch   11/21   train_loss = 3.541
Epoch 174 Batch   16/21   train_loss = 3.586
Epoch 175 Batch    0/21   train_loss = 3.606
Epoch 175 Batch    5/21   train_loss = 3.622
Epoch 175 Batch   10/21   train_loss = 3.543
Epoch 175 Batch   15/21   train_loss = 3.501
Epoch 175 Batch   20/21   train_loss = 3.663
Epoch 176 Batch    4/21   train_loss = 3.513
Epoch 176 Batch    9/21   train_loss = 3.652
Epoch 176 Batch   14/21   train_loss = 3.314
Epoch 176 Batch   19/21   train_loss = 3.494
Epoch 177 Batch    3/21   train_loss = 3.490
Epoch 177 Batch    8/21   train_loss = 3.489
Epoch 177 Batch   13/21   train_loss = 3.446
Epoch 177 Batch   18/21   train_loss = 3.474
Epoch 178 Batch    2/21   train_loss = 3.570
Epoch 178 Batch    7/21   train_loss = 3.617
Epoch 178 Batch   12/21   train_loss = 3.445
Epoch 178 Batch   17/21   train_loss = 3.489
Epoch 179 Batch    1/21   train_loss = 3.606
Epoch 179 Batch    6/21   train_loss = 3.440
Epoch 179 

...

Epoch 784 Batch    6/21   train_loss = 1.715
Epoch 784 Batch   11/21   train_loss = 1.803
Epoch 784 Batch   16/21   train_loss = 1.754
Epoch 785 Batch    0/21   train_loss = 1.748
Epoch 785 Batch    5/21   train_loss = 1.794
Epoch 785 Batch   10/21   train_loss = 1.763
Epoch 785 Batch   15/21   train_loss = 1.813
Epoch 785 Batch   20/21   train_loss = 1.974
Epoch 786 Batch    4/21   train_loss = 1.765
Epoch 786 Batch    9/21   train_loss = 1.968
Epoch 786 Batch   14/21   train_loss = 1.749
Epoch 786 Batch   19/21   train_loss = 1.989
Epoch 787 Batch    3/21   train_loss = 1.820
Epoch 787 Batch    8/21   train_loss = 1.815
Epoch 787 Batch   13/21   train_loss = 1.704
Epoch 787 Batch   18/21   train_loss = 1.837
Epoch 788 Batch    2/21   train_loss = 1.828
Epoch 788 Batch    7/21   train_loss = 1.704
Epoch 788 Batch   12/21   train_loss = 1.833
Epoch 788 Batch   17/21   train_loss = 1.885
Epoch 789 Batch    1/21   train_loss = 1.783
Epoch 789 Batch    6/21   train_loss = 1.827
Epoch 789 

Epoch 827 Batch   18/21   train_loss = 2.333
Epoch 828 Batch    2/21   train_loss = 2.305
Epoch 828 Batch    7/21   train_loss = 2.317
Epoch 828 Batch   12/21   train_loss = 2.660
Epoch 828 Batch   17/21   train_loss = 2.656
Epoch 829 Batch    1/21   train_loss = 2.508
Epoch 829 Batch    6/21   train_loss = 2.841
Epoch 829 Batch   11/21   train_loss = 2.666
Epoch 829 Batch   16/21   train_loss = 2.841
Epoch 830 Batch    0/21   train_loss = 2.315
Epoch 830 Batch    5/21   train_loss = 2.742
Epoch 830 Batch   10/21   train_loss = 2.498
Epoch 830 Batch   15/21   train_loss = 2.716
Epoch 830 Batch   20/21   train_loss = 2.759
Epoch 831 Batch    4/21   train_loss = 2.437
Epoch 831 Batch    9/21   train_loss = 2.785
Epoch 831 Batch   14/21   train_loss = 2.763
Epoch 831 Batch   19/21   train_loss = 2.730
Epoch 832 Batch    3/21   train_loss = 2.615
Epoch 832 Batch    8/21   train_loss = 2.722
Epoch 832 Batch   13/21   train_loss = 4.143
Epoch 832 Batch   18/21   train_loss = 3.932
Epoch 833 

Epoch 871 Batch    9/21   train_loss = 2.544
Epoch 871 Batch   14/21   train_loss = 2.168
Epoch 871 Batch   19/21   train_loss = 2.223
Epoch 872 Batch    3/21   train_loss = 2.229
Epoch 872 Batch    8/21   train_loss = 2.307
Epoch 872 Batch   13/21   train_loss = 2.101
Epoch 872 Batch   18/21   train_loss = 2.202
Epoch 873 Batch    2/21   train_loss = 2.149
Epoch 873 Batch    7/21   train_loss = 2.216
Epoch 873 Batch   12/21   train_loss = 2.345
Epoch 873 Batch   17/21   train_loss = 2.387
Epoch 874 Batch    1/21   train_loss = 2.411
Epoch 874 Batch    6/21   train_loss = 2.304
Epoch 874 Batch   11/21   train_loss = 2.384
Epoch 874 Batch   16/21   train_loss = 2.386
Epoch 875 Batch    0/21   train_loss = 2.200
Epoch 875 Batch    5/21   train_loss = 2.286
Epoch 875 Batch   10/21   train_loss = 2.041
Epoch 875 Batch   15/21   train_loss = 2.435
Epoch 875 Batch   20/21   train_loss = 2.446
Epoch 876 Batch    4/21   train_loss = 2.156
Epoch 876 Batch    9/21   train_loss = 2.494
Epoch 876 

Epoch 915 Batch    0/21   train_loss = 1.804
Epoch 915 Batch    5/21   train_loss = 1.959
Epoch 915 Batch   10/21   train_loss = 1.761
Epoch 915 Batch   15/21   train_loss = 1.968
Epoch 915 Batch   20/21   train_loss = 2.192
Epoch 916 Batch    4/21   train_loss = 1.783
Epoch 916 Batch    9/21   train_loss = 2.082
Epoch 916 Batch   14/21   train_loss = 1.774
Epoch 916 Batch   19/21   train_loss = 1.885
Epoch 917 Batch    3/21   train_loss = 1.875
Epoch 917 Batch    8/21   train_loss = 1.892
Epoch 917 Batch   13/21   train_loss = 1.843
Epoch 917 Batch   18/21   train_loss = 1.839
Epoch 918 Batch    2/21   train_loss = 1.834
Epoch 918 Batch    7/21   train_loss = 1.864
Epoch 918 Batch   12/21   train_loss = 1.967
Epoch 918 Batch   17/21   train_loss = 1.916
Epoch 919 Batch    1/21   train_loss = 1.802
Epoch 919 Batch    6/21   train_loss = 1.851
Epoch 919 Batch   11/21   train_loss = 1.921
Epoch 919 Batch   16/21   train_loss = 1.908
Epoch 920 Batch    0/21   train_loss = 1.864
Epoch 920 

Epoch 958 Batch   12/21   train_loss = 2.061
Epoch 958 Batch   17/21   train_loss = 2.039
Epoch 959 Batch    1/21   train_loss = 2.053
Epoch 959 Batch    6/21   train_loss = 2.006
Epoch 959 Batch   11/21   train_loss = 2.132
Epoch 959 Batch   16/21   train_loss = 2.182
Epoch 960 Batch    0/21   train_loss = 1.974
Epoch 960 Batch    5/21   train_loss = 2.254
Epoch 960 Batch   10/21   train_loss = 1.833
Epoch 960 Batch   15/21   train_loss = 2.182
Epoch 960 Batch   20/21   train_loss = 2.282
Epoch 961 Batch    4/21   train_loss = 1.929
Epoch 961 Batch    9/21   train_loss = 2.372
Epoch 961 Batch   14/21   train_loss = 2.028
Epoch 961 Batch   19/21   train_loss = 2.138
Epoch 962 Batch    3/21   train_loss = 2.125
Epoch 962 Batch    8/21   train_loss = 2.218
Epoch 962 Batch   13/21   train_loss = 2.225
Epoch 962 Batch   18/21   train_loss = 2.155
Epoch 963 Batch    2/21   train_loss = 2.046
Epoch 963 Batch    7/21   train_loss = 2.147
Epoch 963 Batch   12/21   train_loss = 2.135
Epoch 963 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

print(tf.__version__)

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

1.2.0


## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [2]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    input_tensor = loaded_graph.get_tensor_by_name('input:0')
    initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    final_state = loaded_graph.get_tensor_by_name('final_state:0')
    probs = loaded_graph.get_tensor_by_name('probs:0')
    return input_tensor, initial_state, final_state, probs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [3]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    return int_to_vocab[np.argmax(probabilities)]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [4]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        # pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        
        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: power off?(.
moe_szyslak:(excited) i know.
(to) you know.
homer_simpson:(to) i know to presidential air friend on the ass.
jacques: hey, i million be start.
homer_simpson: sorry, say. off!
marge_simpson:(hits) well, moe, i can had over be wings...
marge_simpson:(quickly)(guess i aerosmith do.
homer_simpson:(drunk) what's you get me.
moe_szyslak:(tentative) boy, ya out 'em.
moe_szyslak:(tentative) all my only) 'cause very lot just an compliment.
homer_simpson:(backing) aren't goodbye comes life.
homer_simpson: ol', homer, i should be thought they're school illegal live.
homer_simpson:(encouraging away) well, there's sorry. new calls around moe's the door.
homer_simpson:(sighs snort) my beer's. what--
moe_szyslak:(accusing chuckle) ooh! somebody you get.
homer_simpson:(to) uh-oh, tha, tell, no, come, okay


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.